In [1]:
import nltk
import spacy
import numpy as np 
import pandas as pd
import os
import pickle

In [2]:
os.chdir('Training  Data')

In [3]:
labels = pd.read_csv('subtaskA_answers_all.csv',header=None)

In [4]:
labels = labels[1].values

In [5]:
data = pd.read_csv('subtaskA_data_all.csv')

In [6]:
data.head()

,id,sent0,sent1
0,0,He poured orange juice on his cereal.,He poured milk on his cereal.
1,1,He drinks apple.,He drinks milk.
2,2,Jeff ran a mile today,"Jeff ran 100,000 miles today"
3,3,A mosquito stings me,I sting a mosquito
4,4,A niece is a person.,A giraffe is a person.


# Preprocessor

In [7]:
nlp = spacy.load('en_core_web_md')

In [8]:
from nltk.tokenize import RegexpTokenizer
import re
from nltk.corpus import wordnet as wn
from difflib import SequenceMatcher

In [9]:
def convert_plural_tense(sent):
    text = sent.replace('-','_')
    text = text.lower()
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    
    wnl = nltk.WordNetLemmatizer()    
    output = []
    # if known pos tag
    #ls = [('running', 'n'), ('helping', 'n'), ('cooks', 'n'), ('finds','n')]
    #output = [wnl.lemmatize(word, pos) for word, pos in ls]
    for w in text:
        syn = wn.synsets(w)
        if len(syn):
            pos = syn[0].pos()
            output.append(wnl.lemmatize(w, pos))
        else:
            output.append(wnl.lemmatize(w))
    return output

In [10]:
def tokenize(sent):
    text = sent.replace('-','_')
    text = text.lower()
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    
    return text

In [11]:
def tokenizer(sent1, sent2):
    li1 = [tk.text for tk in sent1]
    li2 = [tk.text for tk in sent2]
    #dif21 = [i for i in li1 + li2 if i not in li1] 
    #dif12 = [i for i in li1 + li2 if i not in li2] 
    (dif12, dif21), (masked1, masked2) = my_get_opcodes(li1,li2)
    #print(dif12)
    #print(dif21)
    spe1 = []
    spe2 = []
    #print(sent1.text)
    #print(sent2.text)
    #print(dif12)
    get_special_word_list(sent1.text, dif12, [], spe1, 1)
    get_special_word_list(sent2.text, dif21, [], spe2, 1)
    return spe1, spe2, " ".join(masked1), " ".join(masked2)

def my_get_opcodes(a, b):
    output = [[],[]]
    masked_output = [[],[]]
    #print(a, b)
    s = SequenceMatcher(None, a, b)
    for tag, i1, i2, j1, j2 in s.get_opcodes():
        #print('{:7}   a[{}:{}] --> b[{}:{}] {!r:>8} --> {!r}'.format(tag, i1, i2, j1, j2, a[i1:i2], b[j1:j2]))
        if tag!='equal':
            output[0]+=a[i1:i2]
            output[1]+=b[j1:j2]
            #masked_output[0]+=['[MASK]']
            #masked_output[1]+=['[MASK]']
        #else:
            #masked_output[0]+=a[i1:i2]
            #masked_output[1]+=b[j1:j2]
    return output, masked_output


def get_special_word_list(sent, dif_a, dif_b, spe, lv):
    #print('-----------------',lv,'----------------')
    if not dif_a:
        return False
    if " ".join(dif_a) in sent:
        #print(lv,'dif_a',dif_a, 'dif_b',dif_b)
        spe.append(" ".join(dif_a))
        dif_a.clear()
        #print(dif_a)
    else:
        #print('not found', lv, dif_a, dif_b)
        dif_a_n = dif_a[:-1]
        dif_b_n = dif_a[-1:]+dif_b
        dif_a = dif_a_n
        dif_b = dif_b_n
        finished = get_special_word_list(sent, dif_a, dif_b, spe,lv+1)
        #print('- A',finished)        
        #print('***spe***',lv, spe, dif_a, dif_b)
        if finished:
            dif_a.clear()
            dif_b.clear()

        finished = get_special_word_list(sent, dif_b, [], spe,lv+1)
        if finished:
            dif_a.clear()
            dif_b.clear()
        #print('-- B', finished)
        #print('***spe***',lv, spe, dif_a, dif_b)
        if dif_b == []:
            return True

In [12]:
def prepare_data(new_sent0, new_sent1):
    output = []
    for i, (sent0, sent1) in enumerate(zip(new_sent0, new_sent1)):
        spec0, spec1, mask1, mask2 = tokenizer(nlp(sent0),nlp(sent1))
        gt = labels[i]
        if labels[i] == 0:
            for spec in spec0:
                output.append({'id':i, 'special':spec,'whole':sent0,'label':0, 'masked':mask1})
            for spec in spec1:
                output.append({'id':i, 'special':spec,'whole':sent1,'label':1,'masked':mask2})
        else:
            for spec in spec0:
                output.append({'id':i, 'special':spec,'whole':sent0,'label':1, 'masked': mask1})
            for spec in spec1:
                output.append({'id':i, 'special':spec,'whole':sent1,'label':0, 'masked':mask2})
    return pd.DataFrame(output)

In [13]:
sample0 = data['sent0'].values
sample1 = data['sent1'].values
new_sent0 = [" ".join(convert_plural_tense(s)) for s in sample0]
new_sent1 = [" ".join(convert_plural_tense(s)) for s in sample1]

In [14]:
df_train = prepare_data(new_sent0, new_sent1)

In [35]:
df_train.to_csv('train_new.csv')

In [17]:
for idx in range(10000):
    if idx not in df_train['id'].unique():
        print(idx)

248
1539
4075
4121
4313
6398
7700
9622


In [15]:
len(df_train['id'].unique())

9992

In [49]:
#df_train['masked'] = np.array(len(df_train),dtype='str')
for i, row in df_train.iterrows():
    masked = row['whole'].replace(' '+row['special'], ' [MASK] ')
    df_train.at[i, 'masked'] = masked
df_train.to_csv('train_masked_new.csv')

In [20]:
data.loc[data['id']==248]

,id,sent0,sent1
248,248,Potatos is the plural of potato,Potatoes is the plural of potato


In [18]:
df_train.loc[df_train['id']==248]

,id,special,whole,label,masked


In [51]:
len(df_train['id'].unique())

9995

In [56]:
df_train['masked'] = np.array(len(df_train),dtype='str')
for i, row in df_train.iterrows():
    masked = row['whole'].replace(' '+row['special'], ' [MASK]')
    df_train.at[i, 'masked'] = masked
df_train.to_csv('train_masked_no_lemma.csv')

In [57]:
df_train.head()

,id,special,whole,label,masked
0,0,orange juice,he poured orange juice on his cereal,0,he poured [MASK] on his cereal
1,0,milk,he poured milk on his cereal,1,he poured [MASK] on his cereal
2,1,apple,he drinks apple,0,he drinks [MASK]
3,1,milk,he drinks milk,1,he drinks [MASK]
4,2,a mile,jeff ran a mile today,1,jeff ran [MASK] today
